# 7-Phase equilibria: MELTS liquid model with ZrSiO<sub>4</sub>

Associated solution model:

| Components: | Species: | Reaction: | Variable: | Component: | Species X: |
| :---------- | :------- | :-------- | :-------- | :--------- | :--------- |
| SiO<sub>2</sub> | SiO<sub>2</sub> | | | $n_1$ | $X_1$ |
| TiO<sub>2</sub> | TiO<sub>2</sub> | | $r_1$ | $n_2$ | $X_2$ |
| Al<sub>2</sub>O<sub>3</sub> | Al<sub>2</sub>O<sub>3</sub> | | $r_2$ | $n_3$ | $X_3$ |
| Fe<sub>2</sub>O<sub>3</sub> | Fe<sub>2</sub>O<sub>3</sub> | | $r_3$ | $n_4$ | $X_4$ |
| MgCr<sub>2</sub>O<sub>4</sub> | MgCr<sub>2</sub>O<sub>4</sub> | | $r_4$ | $n_5$ | $X_5$ |
| Fe<sub>2</sub>SiO<sub>4</sub> | Fe<sub>2</sub>SiO<sub>4</sub> | | $r_5$ | $n_6$ | $X_6$ |
| MnSi<sub>1/2</sub>O<sub>2</sub> | MnSi<sub>1/2</sub>O<sub>2</sub> | | $r_6$ | $n_7$ | $X_7$ |
| Mg<sub>2</sub>SiO<sub>4</sub> | Mg<sub>2</sub>SiO<sub>4</sub> | | $r_7$ | $n_8$ | $X_8$ |
| NiSi<sub>1/2</sub>O<sub>2</sub> | NiSi<sub>1/2</sub>O<sub>2</sub>  | | $r_8$ | $n_9$ | $X_9$ |
| CoSi<sub>1/2</sub>O<sub>2</sub> | CoSi<sub>1/2</sub>O<sub>2</sub> | | $r_9$ | $n_{10}$ | $X_{10}$ |
| CaSiO<sub>3</sub> | CaSiO<sub>3</sub> | | $r_{10}$ | $n_{11}$ | $X_{11}$ |
| Na<sub>2</sub>SiO<sub>3</sub> | Na<sub>2</sub>SiO<sub>3</sub> | | $r_{11}$ | $n_{12}$ | $X_{12}$ |
| KAlSiO<sub>4</sub> | KAlSiO<sub>4</sub> | | $r_{12}$ | $n_{13}$ | $X_{13}$ |
| Ca<sub>3</sub>(PO<sub>4</sub>)<sub>2</sub> | Ca<sub>3</sub>(PO<sub>4</sub>)<sub>2</sub> | | $r_{13}$ | $n_{14}$ | $X_{14}$ |
| H<sub>2</sub>O | H<sub>2</sub>O | | $r_{14}$ | $n_{15}$ | $X_{15}$ |
| CO<sub>2</sub> | CO<sub>2</sub> | | $r_{15}$ | $n_{16}$ | $X_{16}$ |
| ZrSiO<sub>4</sub> | ZrSiO<sub>4</sub> | | $r_{16}$ | $n_{17}$ | $X_{17}$ |
| | CaCO<sub>3</sub> | CO<sub>2</sub> + CaSiO<sub>3</sub> = CaCO<sub>3</sub> + SiO<sub>2</sub> | $s_1$ | | $X_{18}$ |
| | Na<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> | ZrSiO<sub>4</sub> + 2Na<sub>2</sub>SiO<sub>3</sub> = Na<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> + SiO<sub>2</sub> | $s_2$ | | $X_{19}$ |
| | K<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> | ZrSiO<sub>4</sub> + 4KAlSiO<sub>4</sub> = K<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> + 2Al<sub>2</sub>O<sub>3</sub> + 3SiO<sub>2</sub> | $s_3$ | | $X_{20}$ |

## Gibbs energy minimization (fixed T, P, bulk composition)
Closed system; crystallization of a rhyolitic liquid using rhyolite-MELTS 1.1 + Zr-model

In [ ]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import sys
from thermoengine import core, phases, model, equilibrate
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt
import scipy as sci
%matplotlib inline

In [ ]:
model_working_dir = "working"
%cd {model_working_dir}
from pyximport import install
install()
import rMELTS_ZR
import zirconium_minerals
modelDB_ZR_min = model.Database(database="CoderModule", calib=True, 
                             phase_tuple=('zirconium_minerals', {
                                 'Zrc':['zircon','pure'],
                                 'Bdd':['baddeleyite','pure']
                             }))
modelDB_ZR_liq = model.Database(database="CoderModule", calib=True, 
                             phase_tuple=('rMELTS_ZR', {
                                 'liq':['Liquid', 'solution']
                             }))
%cd ..

In [ ]:
modelDB_ZR_min.phase_info

In [ ]:
modelDB_ZR_liq.phase_info

## Read in parameter estimates from first calibration notebook

In [ ]:
import pickle
with open('calib_params.pickle', 'rb') as f:
    params_stored = pickle.load(f)
for i,x in enumerate(params_stored):
    rMELTS_ZR.cy_Liquid_rMELTS_ZR_set_param_value(i, x) 

## Create phases for equilibrium assemblages

In [ ]:
src_obj = core.get_src_object('EquilibrateUsingMELTSv102')
modelDB = model.Database()
Liquid = modelDB_ZR_liq.get_phase('liq')
Feldspar = modelDB.get_phase('Fsp')
Quartz = modelDB.get_phase('Qz')
Spinel = modelDB.get_phase('SplS')
Opx = modelDB.get_phase('Opx')
RhomOx = modelDB.get_phase('Rhom')
Zircon = modelDB_ZR_min.get_phase('Zrc')
Baddeleyite = modelDB_ZR_min.get_phase('Bdd')

The MELTS 1.0.2 water model.

In [ ]:
Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)

The MELTS 1.1.x fluid (H<sub>2</sub>O, CO<sub>2</sub>) model.

In [ ]:
Fluid = phases.SolutionPhase('FluidDuan','Fld')

## Define elements in system and phases in system

In [ ]:
elm_sys = ['H','C','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni','Zr']
#phs_sys = [Liquid, Feldspar, Quartz, Spinel, Opx, RhomOx, Zircon, Baddeleyite, Water]
phs_sys = [Liquid, Feldspar, Quartz, Spinel, Opx, RhomOx, Zircon, Baddeleyite, Fluid]

## Composition of the system
This is a high-silica rhyolite

In [ ]:
grm_oxides = {
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    5.5,
    'CO2':    0.0005
}
Zr_ppm = 100

Cast this composition as moles of elements for input to the Equilibrate class

In [ ]:
tot_grm_oxides = 0.0
for key in grm_oxides.keys():
    tot_grm_oxides += grm_oxides[key]
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
mol_elm = core.chem.mol_oxide_to_elem(mol_oxides)
elm_ref = np.zeros(107)
for i,sym in enumerate(
    ['Si', 'Ti', 'Al', 'Fe', 'Cr', 'Mn', 'Mg', 'Ni', 'Co', 'Ca', 'Na', 'K', 'P', 'H', 'C', 'O']):
    index = core.chem.PERIODIC_ORDER.tolist().index(sym)
    elm_ref[index] = mol_elm[i]
#
elm = np.copy(elm_ref)
wtZrO2 = 123.218*Zr_ppm/91.224/1000000.0
wtZrO2 *= tot_grm_oxides + wtZrO2
elm[core.chem.PERIODIC_ORDER.tolist().index('Zr')] += wtZrO2/123.218
elm[core.chem.PERIODIC_ORDER.tolist().index('O')]  += wtZrO2*2.0/123.218
blk_cmp = []
for x in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(x)
    blk_cmp.append(elm[index])
blk_cmp = np.array(blk_cmp)

## Instantiate class instance and run calculation

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [ ]:
t =  762.0 + 273.15
p = 1750.0
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0, stats=False)
state.print_state()

Set up a function to help with plotting results

In [ ]:
phs_mass = []
phs_name = []
temperatures = []
def plot_data():
    global phs_mass, phs_name, temperatures
    phs_state = state.properties()
    temperatures.append(state.temperature - 273.15)
    masses = []
    names = []
    for key in phs_state.keys():
        if phs_state[key] == 'stable':
            names.append(key)
            masses.append(state.properties(phase_name=key, props='Mass'))
    phs_name.append(names)
    phs_mass.append(masses)
plot_data()

Run a sequence ...

In [ ]:
for i in range(1,61):
    state = equil.execute(t-i/5.0, p, state=state, debug=0, stats=False)
    state.print_state()
    plot_data()

Massage output into a suitable state for plotting

In [ ]:
temperatures = np.array(temperatures)
phs_name_unique = []
for lst in phs_name:
    phs_name_unique += lst
phs_name_unique = list(sorted((set(phs_name_unique))))
phs_name_unique.remove('System')
phs_mass_unique = []
for i,lst in enumerate(phs_mass):
    masses = np.zeros(len(phs_name_unique))
    for j,name in enumerate(phs_name[i]):
        if name is not 'System':
            ind = phs_name_unique.index(name)
            masses[ind] = lst[j]
    phs_mass_unique.append(masses)
phs_mass_unique = np.array(phs_mass_unique)
zr_onset = 0.0
zr_ind = phs_name_unique.index('zircon')
for i,tval in enumerate(temperatures):
    if phs_mass_unique[i,zr_ind] > 0.0:
        zr_onset = tval
        break

## Plot results of sequence

In [ ]:
fig = plt.figure(figsize=(15,10))
for i,name in enumerate(phs_name_unique):
    plt.plot(temperatures, phs_mass_unique[:,i], label=name)
plt.plot(np.array([zr_onset, zr_onset]), np.array([0,105]), 'k--', label='zircon liquidus')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.stackplot(temperatures, phs_mass_unique.T, labels=phs_name_unique)
plt.plot(np.array([zr_onset, zr_onset]), np.array([0,105]), 'k--', label='zircon liquidus')
plt.legend(loc='upper right')
plt.title("High-silica rhyolite")
plt.xlabel("Temperature °C")
plt.ylabel("Mass (g)")
plt.xlim(750, 762)
plt.ylim(0,105)
fig.savefig("MELTS-zr-rhyolite.pdf", bbox_inches='tight')
plt.show()